---
### CREATING ARTIST METADATA DATAFRAME

In [2]:
import spotipy
import sys
from pprint import pprint
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

import requests
import json

In [3]:
import spotipy.util as util

In [6]:
CLIENT_ID = ''
CLIENT_SECRET = ''

In [7]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [8]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials('', ''))

In [11]:
#load created dataframe of 2020 only steaming data
NEW_YEAR_STREAM=pd.read_csv('./data/stream_2020.csv', index_col=0)

---
### Grabbing the genres for each artist - Searching through Spotify API return of artists and recording all of the associated genres
- Had to add a lot of conditional statements to handle many instances where there were no genres listed and also skip duplicates - probably a better way to do this so I'll clean it up later

In [14]:
ARTISTS_list=[]


for artist_i in NEW_YEAR_STREAM['artistName'].unique():
    
    if artist_i in ARTISTS_METADATA['ARTIST']:
        #print(artist_i)
        ARTISTS_list.append(list(ARTISTS_METADATA.loc[artist_i].values))
        pass
    else:
        result = sp.search('artist:'+artist_i, type='artist', limit=50)
        genre_i=None
        id_i = None
        p=0
      #  print(len(ARTISTS_list))
        for i in result['artists']['items']:
            if genre_i != None:
                break
            elif (i['name']==artist_i) and (i['genres'] !=[]):
                genre_i=i['genres']
                id_i=i['id']
                ARTISTS_list.append([artist_i, i['genres'], i['id']])
                p=1

        if (genre_i is None) and (p==1):
            #print artists with no associated info
            print(artist_i)
            id_i=i['id']
            ARTISTS_list.append([artist_i, [], id_i])
        elif (genre_i is None):
            id_i=i['id']
            ARTISTS_list.append([artist_i, [], id_i])
    

---
#### Put the artists info into a pandas df for easier use and saving

In [17]:
artist_metadata=pd.DataFrame(ARTISTS_list, columns=['ARTIST', 'GENRES', 'ID'])
artist_metadata.index=artist_metadata['ARTIST']

### SAVE ARTIST_META DATAFRAME

In [20]:
artist_metadata.to_json('./data/artists_metadata.json')